In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import math
import time
from scipy.integrate import odeint

In [70]:
# Specifying parameters for PFR and other constants taken Siettos et al.'s work (1998)

T0_s = 440 #440 # inlet temperature

k_0 = 3.34 * (np.power(10,10)) # pre-exponential constant

C_p = 25 # heat capacity of reacting liquid

rho_L = 47 # density of reacting liquid

T_s = 423 # steady-state reactor temperature

u = 2 # volumetric flow rate  (F/A) Superficial velocity

E_by_R = 8600 # activation energy (E/R)

delH_term = -44000 # enthalpy of reaction

U = 25 #Overall heat transfer coeffecient

CA0_s = 1.6 #  steady-state inlet concentration of A

CA_s = 0.11 # steady-state reactor concentration of A

Tc_s = 293 # steady state cooling temperature

At = 0.01 # Area for heating rate equation

A = 0.002  # Area

t_final = 0.05 #0.01 # end time for numerical simulation

t_step = 0.01 #1e-3 # integration time step h_c

P = np.array([[1060, 22], [22, 0.52]]) # a positive definite matrix

length = 1 # total length of reactor

N = 10     # number of points to discretize the reactor

# Euler Method

In [42]:
def PFR_simulation_Euler(u, delH_term, k_0, C_p, rho_L, E_by_R, Tc, U, At, A, t_final, t_step, init_C, init_T):

    # Method of lines approximates the spatial derivative using finite difference method which reults in a set of coupled ODE

    def method_of_lines_C(C, T):
        'coupled ODES at each node point'
        D = -u * np.diff(C) / np.diff(z) - k_0 * np.exp(-E_by_R/T[1:]) * C[1:]    # for first order
        return np.concatenate([[0], D]) #C0 is constant at entrance


    def method_of_lines_T(C, T):
        'coupled ODES at each node point'
        D = -u * np.diff(T) / np.diff(z) + (-delH_term/(rho_L*C_p)) * k_0 * np.exp(-E_by_R/T[1:])* C[1:] + (U/(rho_L*C_p*A)) * At * (Tc - T[1:]) # for first order
        return np.concatenate([[0], D]) #T0 is constant at entrance

    N = 11    # number of points to discretize the reactor length
    z = np.linspace(0, length, N) # discretized length elements

    #initializing arrays
    init_C_A_2_1 = np.zeros(N)
    init_T_2_1 = np.zeros(N)

    init_C_A_2_2 = np.zeros(N)
    init_T_2_2 = np.zeros(N)

    init_C_A_3 = np.zeros(N)
    init_T_3 = np.zeros(N)

    C_A_3 = np.zeros(N)
    T_3 = np.zeros(N)


    dCAdt1 = method_of_lines_C(init_C, init_T)
    dTdt1 = method_of_lines_T(init_C, init_T)

    for i in range(len(init_C)):
        init_C_A_2_1[i] = init_C[i] + dCAdt1[i] * t_step / 2
        init_T_2_1[i] = init_T[i] + dTdt1[i] * t_step / 2

    dCAdt2_1 = method_of_lines_C(init_C_A_2_1, init_T_2_1)
    dTdt2_1 = method_of_lines_T(init_C_A_2_1, init_T_2_1)

    for i in range(len(init_C)):
        init_C_A_2_2[i] = init_C[i] + dCAdt2_1[i] * t_step / 2
        init_T_2_2[i] = init_T[i] + dTdt2_1[i] * t_step / 2

    dCAdt2_2 = method_of_lines_C(init_C_A_2_2, init_T_2_2)
    dTdt2_2 = method_of_lines_T(init_C_A_2_2, init_T_2_2)

    for i in range(len(init_C)):
        init_C_A_3[i] = init_C[i] + dCAdt2_2[i] * t_step / 2
        init_T_3[i] = init_T[i] + dTdt2_2[i] * t_step / 2

    dCAdt3 = method_of_lines_C(init_C_A_3, init_T_3)
    dTdt3 = method_of_lines_T(init_C_A_3, init_T_3)

    dCAdt2 = np.add(dCAdt2_1,dCAdt2_2)
    dCAdt2 = np.divide(dCAdt2,2)

    dTdt2 = np.add(dTdt2_1,dTdt2_2)
    dTdt2 = np.divide(dTdt2,2)

    for i in range(len(init_C)):
        C_A_3[i] = init_C[i] + t_step / 6 * (dCAdt1[i] + 4*dCAdt2[i] + dCAdt3[i])
        T_3[i] = init_T[i] + t_step / 6 * (dTdt1[i] + 4*dTdt2[i] + dTdt3[i])

    return C_A_3 , T_3

# Data generation (PI-RNN) collocation points

In [36]:
# generating inputs and initial states for PFR

u2_physics_list = np.linspace(100, 300, 20) # u2 is the cooling temp
T_physics_initial = np.linspace(300, 500, 20)  # inlet temperature
CA_physics_initial = np.linspace(0.5, 3, 20)  # inlet concentration

In [37]:
# Grouping the initial state vectors

T_physics_start = list()
CA_physics_start = list()

for T in T_physics_initial:
    for CA in CA_physics_initial:
        CA_physics_start.append(CA)
        T_physics_start.append(T)

print("number of initial conditions: {}".format(len(CA_physics_start)))

# convert to np.arrays
CA_physics_start = np.array([CA_physics_start])
T_physics_start = np.array([T_physics_start])
x_physics_original = np.concatenate((CA_physics_start.T, T_physics_start.T), axis=1)  # every row is a pair of initial states within stability region
print("shape of x_physics_original is {}".format(x_physics_original.shape))

number of initial conditions: 400
shape of x_physics_original is (400, 2)


In [40]:
# Steady state values of concnetration and temperature for this system is as given below

CA_ss = [1.6, 1.5111947036559625, 1.417387640197116, 1.3184219134355777, 1.2143091606184162, 1.105331317367263, 0.9921731706476451, 0.8760742913774546, 0.7589642805672522, 0.6435103342443346, 0.5329795659848772]
CA_ss = np.array(CA_ss)

T_ss = [440, 442.537980203289, 445.24937842606676, 448.1391353316486, 451.20586746040226, 454.43809815470087, 457.80935816302986, 461.2725883629541, 464.75519617827314, 468.1574151982801, 471.3575957995257]
T_ss = np.array(T_ss)

In [71]:
# get X and y data for the model

CA_physics_output = list()
T_physics_output = list()
CA_physics_input = list()
T_physics_input = list()
Tc_physics_input = list()


for num_id, u2 in enumerate(u2_physics_list):
    print(f"{num_id + 1} out of {u2_physics_list.shape[0]}")    #just to count and keep track
    Tc = u2 + Tc_s

    for C_A_initial, T_initial in x_physics_original:
        Tc_physics_input.append(u2)
        CA_physics_input.append(C_A_initial)
        T_physics_input.append(T_initial)

        N = 11     # number of points to discretize the reactor length
        z = np.linspace(0, length, N) # discretized length elements

        # Here, we initilise with the stead state values to validate our simulation model
        # init_C = CA_ss
        # init_T = T_ss


        # PFR simulation for a certain initial condition that is not at steady state

        init_C = np.zeros(N)    # Concentration in reactor at t = 0
        init_C[0] = C_A_initial          # concentration at entrance
        init_T = np.zeros(N)    # T in reactor at t = 0
        for i in range(len(init_T)):
           if i == 0:
               init_T[i] = T_initial
           else:
               init_T[i] = 200   # at room temperature and scaled with Steady State value

        C_A_list = [init_C]
        T_list = [init_T]

        for _ in range(int(t_final / t_step)):

            CA_next, T_next = PFR_simulation_Euler(u, delH_term, k_0, C_p, rho_L, E_by_R, Tc, U, At, A, t_final, t_step, init_C, init_T)
            C_A_list.append(CA_next)
            T_list.append(T_next)
            init_C = CA_next
            init_T = T_next

        CA_physics_output.append(C_A_list)
        T_physics_output.append(T_list)

1 out of 20
2 out of 20
3 out of 20
4 out of 20
5 out of 20
6 out of 20
7 out of 20
8 out of 20
9 out of 20
10 out of 20
11 out of 20
12 out of 20
13 out of 20
14 out of 20
15 out of 20
16 out of 20
17 out of 20
18 out of 20
19 out of 20
20 out of 20


In [87]:

for i in CA_physics_output[100]:
    print(i)

[0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
[5.00000000e-01 9.21500000e-02 7.38333333e-03 4.50000000e-04
 1.66666667e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[5.00000000e-01 1.67316745e-01 2.90284144e-02 3.34659942e-03
 2.88797222e-04 1.85361111e-05 8.67222222e-07 2.94444444e-08
 5.55555556e-10 0.00000000e+00 0.00000000e+00]
[5.00000000e-01 2.28630004e-01 5.94275587e-02 1.04212297e-02
 1.36713389e-03 1.41222024e-04 1.17225367e-05 7.89919221e-07
 4.31616666e-08 1.87194444e-09 6.21296296e-11]
[5.00000000e-01 2.78640894e-01 9.46186192e-02 2.21958765e-02
 3.92079155e-03 5.50602174e-04 6.34741546e-05 6.12220025e-06
 4.99640669e-07 3.46647654e-08 2.04189244e-09]
[5.00000000e-01 3.19423809e-01 1.31801690e-01 3.84599569e-02
 8.53132959e-03 1.51290471e-03 2.21879980e-04 2.75277625e-05
 2.93353046e-06 2.71240647e-07 2.18924675e-08]


In [88]:
for i in T_physics_output[100]:
    print(i)

[352.63157895 200.         200.         200.         200.
 200.         200.         200.         200.         200.
 200.        ]
[352.63157895 228.30713626 202.45666927 200.34248118 200.21031589
 200.20522817 200.20522817 200.20522817 200.20522817 200.20522817
 200.20522817]
[352.63157895 251.37175216 209.25318947 201.42979379 200.49823709
 200.41588762 200.41050249 200.41024709 200.41023827 200.4102381
 200.4102381 ]
[352.63157895 270.1647789  218.69780586 203.78647974 201.0311344
 200.65801913 200.6185991  200.61527059 200.61504319 200.61503061
 200.61503006]
[352.63157895 285.47739857 229.57654588 207.56682608 202.01163649
 200.9870247  200.83890735 200.82146633 200.81975621 200.81961476
 200.81960483]
[352.63157895 297.95451048 241.03429951 212.70187695 203.61484715
 201.48348069 201.09136209 201.03232418 201.02485223 201.02404328
 201.0239675 ]


# Collating Input and Output for RNN

In [ ]:
# collate input for RNN for physics loss

Tc_physics_input = np.array(Tc_physics_input)
Tc_physics_input = Tc_physics_input.reshape(-1,1,1)


CA_physics_input = np.array(CA_physics_input)
CA_physics_input = CA_physics_input.reshape(-1,1,1)


T_physics_input = np.array(T_physics_input)
T_physics_input = T_physics_input.reshape(-1,1,1)


RNN_physics_input_temp = np.concatenate((Tc_physics_input, CA_physics_input, T_physics_input), axis=2)

"""
    the input to RNN is in the shape [number of samples x timestep x variables], and the input variables are same for every
    time step
"""

RNN_physics_input_temp = RNN_physics_input_temp.repeat(6, axis=1)
print("RNN_physics_input_temp shape is {}".format(RNN_physics_input_temp.shape))

RNN_physics_input_temp shape is (8000, 6, 3)


In [ ]:
############################## collate output for RNN ####################################################

CA_physics_output = np.array(CA_physics_output)
CA_physics_output = CA_physics_output.reshape(-1, 6, 10)

T_physics_output = np.array(T_physics_output)
T_physics_output = T_physics_output.reshape(-1, 6, 10)

RNN_physics_output = np.concatenate((CA_physics_output, T_physics_output), axis=2)
print("RNN_physics_output shape is {}".format(RNN_physics_output.shape))  # output shape: number of samples x timestep x variables


RNN_physics_output shape is (8000, 6, 20)


In [ ]:
############################### Split into Training, Test, and Validation Data ################################

X_train, X_test, y_train, y_test = train_test_split(RNN_physics_input_temp, RNN_physics_output, test_size=0.2, random_state=123)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123) # 0.25 x 0.8 = 0.2

print(f"X_train shape is {X_train.shape}, X_val shape is {X_val.shape}, X_test shape is {X_test.shape}")
print(f"y_train shape is {y_train.shape}, y_val shape is {y_val.shape}, y_test shape is {y_test.shape}")

X_train shape is (4800, 6, 3), X_val shape is (1600, 6, 3), X_test shape is (1600, 6, 3)
y_train shape is (4800, 6, 20), y_val shape is (1600, 6, 20), y_test shape is (1600, 6, 20)
